In [6]:
import os
import glob
import json
import numpy as np
from tqdm import tqdm
from scipy import linalg
from scipy.spatial.distance import cdist

# Cố gắng import metrics từ file local, nếu không có thì dùng fallback implementation dưới
try:
    from utils import metrics
    HAS_METRICS = True
except Exception as e:
    print("Không load được metrics.py (sẽ dùng fallback). Lỗi:", e)
    HAS_METRICS = False

# Cấu hình chung
FEATURE_POOLING = "mean"   # options: "mean", "max", "last"
COV_EPS = 1e-6             # regularization cho covariance nếu cần
TOP_K_VALUES = [1, 3, 5]
OUTPUT_RESULTS_JSON = "eval_results.json"

In [7]:
import torch
from os.path import join as pjoin

# Thêm các import bạn đã chỉ ra (chỉnh đường dẫn import nếu cần)
# from models import MotionTransformer        # nếu file models.py có trong PYTHONPATH
# from utils.get_opt import get_opt
# from trainers import DDPMTrainer

# Nếu các import trên bị lỗi do path, chỉnh sys.path trước:
# import sys
# sys.path.append("/home/serverai/ltdoanh/Motion_Diffusion/")  # chỉnh theo repo của bạn

# Config: đường dẫn opt, checkpoint sẽ lấy từ opt (giống code bạn gửi)
OPT_PATH = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/opt.txt"  # chỉnh nếu khác
APPLY_DENORMALIZE = True   # nếu model output được chuẩn hoá và bạn muốn nhân *std + mean

# --- Khởi tạo encoder & trainer giống code bạn đưa ---
# NOTE: nếu bạn đã có encoder/trainer ở scope khác, bạn có thể skip phần khởi tạo
try:
    from models import MotionTransformer
    from utils.get_opt import get_opt
    from trainers import DDPMTrainer
except Exception as e:
    print("Warning: không import được các module model/trainer trực tiếp. Nếu notebook nằm ngoài repo, bổ sung sys.path. Lỗi:", e)

# tạo encoder (cần khớp với config của bạn)
encoder = MotionTransformer(
    input_feats=264,
    num_frames=360,
    num_layers=8,
    latent_dim=512,
    no_clip=False,
    no_eff=False
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
opt = get_opt(OPT_PATH, device)
opt.do_denoise = True

# load mean/std nếu muốn denormalize
mean = None
std = None
try:
    mean = np.load(pjoin(opt.meta_dir, 'mean.npy'))
    std = np.load(pjoin(opt.meta_dir, 'std.npy'))
    print("Loaded mean/std from", opt.meta_dir)
except Exception as e:
    print("Không tìm thấy mean/std hoặc lỗi load:", e)

trainer = DDPMTrainer(opt, encoder)
trainer.load(pjoin(opt.model_dir, opt.which_epoch + '.tar'))

trainer.eval_mode()
trainer.to(opt.device)

print("Model & trainer đã load. Device:", opt.device)

Reading /home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/opt.txt
Loaded mean/std from ./checkpoints/beat/test/meta


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 79.25 GiB of which 12.75 MiB is free. Process 3741889 has 58.55 GiB memory in use. Process 3888573 has 18.95 GiB memory in use. Including non-PyTorch memory, this process has 1.71 GiB memory in use. Of the allocated memory 1.27 GiB is allocated by PyTorch, and 38.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Folder chứa cả .npy và .txt
TEST_FOLDER_NPY = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_testing/npy"
TEST_FOLDER_TXT = "/home/serverai/ltdoanh/Motion_Diffusion/datasets/BEAT_testing/txt"

# Lấy file npy và txt
real_feature_files = sorted(glob.glob(os.path.join(TEST_FOLDER_NPY, "*.npy")))
prompt_files = sorted(glob.glob(os.path.join(TEST_FOLDER_TXT, "*.txt")))

# Kiểm tra số lượng
print(len(real_feature_files), "NPYs")
print(len(prompt_files), "TXT prompts")

# Đọc prompt
LIST_OF_TEST_PROMPTS = []
for txt_file in prompt_files:
    with open(txt_file, "r", encoding="utf-8") as f:
        LIST_OF_TEST_PROMPTS.append(f.read().strip())

print("Loaded prompts:", len(LIST_OF_TEST_PROMPTS))

34 NPYs
34 TXT prompts
Loaded prompts: 34


In [ ]:
# Tìm file thật và kiểm tra số lượng
# real_feature_files = sorted(glob.glob(PATH_TO_TEST_NPYS))
n_files = len(real_feature_files)
n_prompts = len(LIST_OF_TEST_PROMPTS)
print(f"Found {n_files} .npy files, {n_prompts} prompts provided.")

if n_files == 0:
    raise FileNotFoundError("Không tìm thấy file .npy nào. Kiểm tra PATH_TO_TEST_NPYS.")
if n_files != n_prompts:
    print("⚠️ Cảnh báo: Số lượng .npy và prompts không khớp.")
    N = min(n_files, n_prompts)
    print(f"Sẽ dùng N = {N} (min of files/prompts). Vui lòng đảm bảo thứ tự tương ứng.")
else:
    N = n_files

Found 34 .npy files, 34 prompts provided.


In [ ]:
# Hàm pooling (T, D) -> (D,)
def pool_segment(segment: np.ndarray, pooling: str = "mean"):
    if segment is None:
        return None
    if segment.ndim == 1:
        return segment
    if pooling == "mean":
        return np.mean(segment, axis=0)
    elif pooling == "max":
        return np.max(segment, axis=0)
    elif pooling == "last":
        return segment[-1]
    else:
        raise ValueError("Unknown pooling: " + str(pooling))

In [ ]:
# Tải real features (với pooling)
real_features_list = []
print(f"Loading and pooling {N} real feature files...")
for i in tqdm(range(N)):
    npy_file = real_feature_files[i]
    seg = np.load(npy_file)
    vec = pool_segment(seg, FEATURE_POOLING)
    real_features_list.append(vec)
real_features = np.vstack(real_features_list)  # (N, D)
print("real_features shape:", real_features.shape)

Loading and pooling 34 real feature files...


100%|██████████| 34/34 [00:00<00:00, 2972.45it/s]

real_features shape: (34, 264)


In [ ]:
# --- Generate features using your trainer (replaces previous generated_features cell) ---
from tqdm import tqdm

# Pooling function (reuse from notebook)
def pool_segment(segment: np.ndarray, pooling: str = "mean"):
    if segment is None:
        return None
    if segment.ndim == 1:
        return segment
    if pooling == "mean":
        return np.mean(segment, axis=0)
    elif pooling == "max":
        return np.max(segment, axis=0)
    elif pooling == "last":
        return segment[-1]
    else:
        raise ValueError("Unknown pooling: " + str(pooling))

# Params: bạn có thể thay đổi pooling, seq_len (frame length) nếu cần
FEATURE_POOLING = "mean"    # "mean" / "max" / "last"
DEFAULT_NUM_FRAMES = getattr(opt, "num_frames", 360)  # nếu opt có num_frames thì dùng
RESULTS_SAVE_DIR = pjoin(opt.result_dir if hasattr(opt, "result_dir") else ".", "")

generated_features_list = []
print(f"Đang sinh {N} mẫu từ model... (pooling={FEATURE_POOLING})")

for i in tqdm(range(N)):
    # lấy prompt tương ứng (LIST_OF_TEST_PROMPTS phải đã được load)
    prompt = LIST_OF_TEST_PROMPTS[i]

    # trainer.generate thường nhận list of captions và lengths -> chúng ta gọi 1 sample mỗi lần
    caption = [prompt]
    # length: dùng DEFAULT_NUM_FRAMES hoặc bạn có ở real data
    try:
        m_lens = torch.LongTensor([int(DEFAULT_NUM_FRAMES)]).to(device)
    except Exception:
        m_lens = torch.LongTensor([360]).to(device)

    with torch.no_grad():
        # Một số trainer.generate trả về list of tensors; giữ nguyên tương thích
        pred_motions = trainer.generate(caption, m_lens, 264)  # 264 là input_feats bạn dùng
        # pred_motions expected: list/tuple of tensors (B, T, D) hoặc tensor (B, T, D)
        if isinstance(pred_motions, (list, tuple)):
            pred = pred_motions[0]
        else:
            pred = pred_motions

        # đưa về CPU numpy
        motion = pred.cpu().numpy()  # shape (T, D) or (B, T, D) ? if B dimension present, take first
        # if batch dim present
        if motion.ndim == 3:
            motion = motion[0]  # (T, D)

    # optional: denormalize if mean/std loaded and flagged
    if APPLY_DENORMALIZE and (mean is not None) and (std is not None):
        try:
            # mean/std could be shape (D,) or (1, D)
            motion = motion * std + mean
        except Exception as e:
            # fallback: if shapes don't broadcast, try per-dim
            if mean.shape[0] == motion.shape[-1]:
                motion = motion * mean + std  # (best-effort, but this is unusual)
            else:
                print(f"Không thể denormalize do shape mismatch: motion {motion.shape}, mean {None if mean is None else mean.shape}, std {None if std is None else std.shape}. Lỗi: {e}")

    # Save raw motion optionally per-sample
    try:
        # lưu mỗi motion nếu cần (comment nếu không muốn)
        np.save(pjoin(RESULTS_SAVE_DIR, f"generated_motion_{i:04d}.npy"), motion)
    except Exception:
        pass

    # Pool to vector
    vec = pool_segment(motion, FEATURE_POOLING)
    generated_features_list.append(vec)

generated_features = np.vstack(generated_features_list)
print("Đã sinh xong generated_features shape:", generated_features.shape)

Generating 34 features from prompts...


100%|██████████| 34/34 [00:00<00:00, 4292.27it/s]

generated_features shape: (34, 256)


In [ ]:
# Fallback implementation cho Frechet Distance (nếu metrics.calculate_frechet_distance không có)
def _fallback_calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    # regularize
    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)
    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)
    dim = mu1.shape[0]
    assert mu2.shape[0] == dim
    # add eps to diagonal if needed
    offset = np.eye(dim) * eps
    covmean, _ = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset), disp=False)
    # numerical fix: if imaginary due to numerical error, take real part
    if np.iscomplexobj(covmean):
        if not np.allclose(np.imag(covmean), 0, atol=1e-3):
            raise ValueError("Imaginary component found in sqrtm result.")
        covmean = np.real(covmean)
    diff = mu1 - mu2
    fid = diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * np.trace(covmean)
    return float(np.real(fid))

In [ ]:
# Compute means & covariances, then FID
mu_real = np.mean(real_features, axis=0)
sigma_real = np.cov(real_features, rowvar=False)
mu_gen = np.mean(generated_features, axis=0)
sigma_gen = np.cov(generated_features, rowvar=False)

print("Computing FID...")
try:
    if HAS_METRICS and hasattr(metrics, "calculate_frechet_distance"):
        fid_score = metrics.calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen)
    else:
        fid_score = _fallback_calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen, eps=COV_EPS)
    print(f"FID = {fid_score:.6f}")
except Exception as e:
    print("Lỗi khi tính FID:", e)
    # last-resort: try with larger eps
    try:
        fid_score = _fallback_calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen, eps=1e-3)
        print("FID (with larger eps) =", fid_score)
    except Exception as e2:
        print("Vẫn lỗi khi tính FID:", e2)
        fid_score = None


Computing FID...
Lỗi khi tính FID: Training and test mean vectors have different lengths
Vẫn lỗi khi tính FID: 


In [ ]:
# Tính Top-K retrieval metrics (R-Precision @1 and Top-K)
print("Computing distance matrix and Top-K accuracies...")
# dist_matrix shape: (N_generated, N_real)
dist_matrix = cdist(generated_features, real_features, metric="euclidean")
sorted_indices = np.argsort(dist_matrix, axis=1)

correct_matches = {k: 0 for k in TOP_K_VALUES}
for i in range(N):
    ground_truth_index = i
    for k in TOP_K_VALUES:
        top_k = sorted_indices[i, :k]
        if ground_truth_index in top_k:
            correct_matches[k] += 1

topk_results = {}
for k in TOP_K_VALUES:
    acc = correct_matches[k] / N
    topk_results[f"top_{k}_acc"] = float(acc)
    print(f"Top-{k} Accuracy: {acc*100:.2f}% {'(R-Precision @1)' if k==1 else ''}")


Computing distance matrix and Top-K accuracies...


ValueError: XA and XB must have the same number of columns (i.e. feature dimension.)

In [ ]:
# Lưu kết quả vào file JSON
results = {
    "N": int(N),
    "feature_pooling": FEATURE_POOLING,
    "fid": None if fid_score is None else float(fid_score),
    "topk": topk_results,
    "notes": "Auto-eval run. Ensure real .npy files and prompts are aligned."
}
with open(OUTPUT_RESULTS_JSON, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)
print("Saved results to", OUTPUT_RESULTS_JSON)


NameError: name 'topk_results' is not defined